In [1]:
# Code source diabetes: Jaques Grobler
# License: BSD 3 clause
# Application to boston dataset and modifications: Jeremy Camponovo

License = """
Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
"""

In [3]:
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import Dropdown, HTML,HTMLMath, interact, widgets, Image, HBox,VBox,Tab,Label,Text,Textarea,Layout,Accordion, Button
from IPython.display import display,clear_output, update_display, Javascript
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt

In [5]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# <center style='color:red;'><b> Le fonctionnement de l'apprentissage machine </b></center>

In [4]:
boston = datasets.load_boston()

feature_descr1 = ["per capita crime rate by town\n", "proportion of residential land zoned for lots over 25,000 sq.ft.\n", "proportion of non-retail business acres per town\n","Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)\n", "nitric oxides concentration (parts per 10 million)\n","average number of rooms per dwelling\n", "proportion of owner-occupied units built prior to 1940\n","weighted distances to five Boston employment centres\n", "index of accessibility to radial highways\n","full-value property-tax rate per $10,000\n", "pupil-teacher ratio by town\n","1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town\n", "% lower status of the population\n"]
feature_name = boston['feature_names']
description = boston['DESCR']

general1 = Accordion()
children1 = []
general2 = Accordion()
children2 = []
general3 = Accordion()
children3 = []
general4 = Accordion()
children4 = []
general5 = Accordion()
children5 = []


# Premier accordéon 1. collecte des données
data1 = "TARGET\nMedian value of owner-occupied homes in $1000's\n\n"
for i in range(len(feature_name)):
    data1+=feature_name[i]+"\n"+feature_descr1[i]+"\n"
texte1 = Textarea(value=data1,disabled=True,layout=Layout(width='95%', height='1000px'))

texte2 = Textarea(value=description,disabled=True,layout=Layout(width='95%', height='1100px'))

tableau=[]
for i in range(len(feature_name)):
    temp=[HTML("<center><b>"+feature_name[i]+"</b></center>",layout=Layout(width='60px', height='25px'))]
    for j in range(15):
        temp.append(HTML("<center>"+str(boston.data[j,i])+"</center>",layout=Layout(width='60px', height='25px')))
    tableau.append(VBox(temp[:]))
temp = [HTML("<center style='color:red;'><b>"+"TARGET"+"</b></center>",layout=Layout(width='60px', height='25px'))]
for j in range(15):
    temp.append(HTML("<center style='color:red;'>"+str(boston.target[j])+"</center>",layout=Layout(width='60px', height='25px')))
tableau.append(VBox(temp[:]))

accordion = Accordion()
accordion.children = [texte1,texte2,HBox(tableau,layout=Layout(width='95%', height='500px'))]
accordion.set_title(0,"Description des clés")
accordion.set_title(1,"Description du jeu de données")
accordion.set_title(2,"exemple de données")
children1.append(accordion)
accordion.selected_index=2

## deuxième accordéon 2.1 separation des sets
## et troisieme accordéon 2.2. Entrainement
boston_xtrain, boston_xtest, boston_ytrain, boston_ytest = train_test_split(boston.data[:,:], boston.target, train_size=0.90)
feature_descr = ["per capita crime rate by town\n", "proportion of residential land zoned\nfor lots over 25,000 sq.ft.", "proportion of non-retail business acres per town\n","Charles River dummy variable\n(= 1 if tract bounds river; 0 otherwise)", "nitric oxides concentration (parts per 10 million)\n","average number of rooms per dwelling\n", "proportion of owner-occupied units built prior to 1940\n","weighted distances to five Boston employment centres\n", "index of accessibility to radial highways\n","full-value property-tax rate per $10,000\n", "pupil-teacher ratio by town\n","1000(Bk - 0.63)^2 where Bk is\nthe proportion of blacks by town", "% lower status of the population\n"]
tab_train = Tab()
tab_train2 = Tab()
tab_test = Tab()
tab_util = Tab()


def change_train():
    global boston_xtrain,boston_xtest,boston_ytrain,boston_ytest,tab_train,plt
    file_train_b = []
    image_train_b = []
    wimage_train_b = []
    children_train_b = []
    file_train2_b = []
    image_train2_b = []
    wimage_train2_b = []
    children_train2_b = []
    comment_train2_b = []
    file_test = []
    image_test = []
    wimage_test = []
    comment_test = []
    children_test = []
    file_util = []
    image_util = []
    wimage_util = []
    comment_util = []
    children_util = []
    i = 0
    regr=[]
    boston_ypred=[]
    boston_ymodel=[]
    for name in feature_name:
        ## image des datas
        plt.clf()
        plt.title(feature_descr[i])
        plt.xlabel(feature_name[i])
        plt.ylabel("TARGET")

        # Training set in gray
        plt.scatter(boston_xtrain[:,i], boston_ytrain,  color='gray',label="entrainement")

        #Testing set in red
        plt.scatter(boston_xtest[:,i], boston_ytest,  color='red',label="test")
        plt.xticks(())
        plt.yticks(())
        mx, Mx = plt.xlim()
        my, My = plt.ylim()
        plt.legend()
        plt.savefig("{}temp.png".format(feature_name[i]))
        plt.clf()
        plt.close()
        
        ## image des modeles
        plt.clf()
        plt.title(feature_descr[i])
        plt.xlabel(feature_name[i])
        plt.ylabel("TARGET")

        # Training set in gray
        plt.scatter(boston_xtrain[:,i], boston_ytrain,  color='gray',label="entrainement")

        #Testing set in red
        #plt.scatter(boston_xtest[:,i], boston_ytest,  color='red',label="test")
        
        regr.append(linear_model.LinearRegression())

        # Train the model using the training sets
        regr[-1].fit(boston_xtrain[:,[i]], boston_ytrain)

        # Make predictions using the testing set
        boston_ypred.append( regr[-1].predict(boston_xtest[:,[i]]))

        # Usefull to plot the regression curve
        xt = np.copy(boston_xtrain[:,[i]])
        xt[0] = mx
        xt[1] = Mx
        boston_ymodel.append(regr[-1].predict(xt))

        # Regression curve in blue
        plt.plot(xt, boston_ymodel[-1], color='blue', linewidth=3, label="modèle")
        
        plt.xticks(())
        plt.yticks(())
        plt.legend()
        plt.xlim([mx,Mx])
        plt.ylim([my,My])
        plt.savefig("{}train.png".format(feature_name[i]))
        plt.clf()
        plt.close()
        
        ## image des tests
        plt.clf()
        plt.title(feature_descr[i])
        plt.xlabel(boston['feature_names'][i])
        plt.ylabel("TARGET")

        #Testing set in red
        plt.scatter(boston_xtest[:,i], boston_ytest,  color='red', label="test")

        # Regression curve in blue
        plt.plot(xt, boston_ymodel[-1], color='blue', linewidth=3, label = "modèle")
        
        plt.xticks(())
        plt.yticks(())
        plt.legend()
        plt.xlim([mx,Mx])
        plt.ylim([my,My])
        plt.savefig("{}test.png".format(feature_name[i]))
        plt.clf()
        plt.close()
        
        ## image utilisation
        plt.clf()
        plt.title(feature_descr[i])
        plt.xlabel(boston['feature_names'][i])
        plt.ylabel("TARGET")

        
        # Regression curve in blue
        plt.plot(xt, boston_ymodel[-1], color='blue', linewidth=3, label = "modele")
        
        x = boston_xtest[0,i]
        y = boston_ytest[0]
        yp = boston_ypred[-1][0]
        plt.plot([x,x,x,mx],[my,y,yp,yp], color='green', linewidth=1, label = "prevision du modèle")
        plt.plot([x,mx],[y,y], color='red', linewidth=1, label = "valeur réelle")
        
        #Testing set in red
        plt.scatter(x, y,  color='red')

        plt.xticks(())
        plt.yticks(())
        plt.legend()
        plt.xlim([mx,Mx])
        plt.ylim([my,My])
        plt.savefig("{}util.png".format(feature_name[i]))
        plt.clf()
        plt.close()
        
        
        ## Tab datas
        file_train_b.append(open("./{}temp.png".format(feature_name[i]),'rb'))
        image_train_b.append(file_train_b[-1].read())
        wimage_train_b.append(Image(
        value=image_train_b[-1],
        format='png',
        width='900px',
        height='450px',
        ))
        
        ## Tab modeles
        file_train2_b.append(open("./{}train.png".format(feature_name[i]),'rb'))
        image_train2_b.append(file_train2_b[-1].read())
        comment_train2_b.append(HTMLMath(value = r"""Equation du modèle: $$TARGET = {:.3f} \times {}{:+.2f}$$""".format(regr[-1].coef_[0],feature_name[i],regr[-1].intercept_)))
        wimage_train2_b.append(VBox([Image(
        value=image_train2_b[-1],
        format='png',
        width='900px',
        height='450px',
        ),comment_train2_b[-1]]))
        
        ## Tab test
        file_test.append(open("./{}test.png".format(feature_name[i]),'rb'))
        image_test.append(file_test[-1].read())
        comment_test.append(HTMLMath(value = r"""Erreur quadratique moyenne (plus c'est faible mieux c'est): {:.1f}
        </br>
        $R^2$ (1 est parfait, plus la valeur est haute mieux c'est): {:.2f}""".format(mean_squared_error(boston_ytest, boston_ypred[-1]),r2_score(boston_ytest, boston_ypred[-1]))))
        wimage_test.append(VBox([Image(
        value=image_test[-1],
        format='png',
        width='900px',
        height='450px',
        ),comment_test[-1]]))
        
        ## Tab utilisation
        file_util.append(open("./{}util.png".format(feature_name[i]),'rb'))
        image_util.append(file_util[-1].read())
        comment_util.append(HTMLMath(value = r"""${} = {:.3f}$
        </br>
        </br>
        Le modèle calculé est $TARGET = {:.3f} \times {}{:+.2f}$ 
        </br>
        On peut donc prévoir $TARGET = {:.2f}$
        </br>
        </br>
        On peut comparer à la valeur réelle: $REAL = {:.2f}$
        </br></br>
        Chaque utilisation ultérieure du modèle peut amener à poursuivre l'entrainement de l'IA et à rendre ses prévisions plus précises. L'utilisation d'autres modèles mathématiques que la droite peuvent aussi améliorer les prévisions""".format(feature_name[i],x,regr[-1].coef_[0],feature_name[i],regr[-1].intercept_,yp,y)))
        wimage_util.append(VBox([Image(
        value=image_util[-1],
        format='png',
        width='900px',
        height='450px',
        ),comment_util[-1]]))
        
        i+=1
    tab_train.children = wimage_train_b
    for i in range(len(feature_name)):
        tab_train.set_title(i, feature_name[i])
    tab_train2.children = wimage_train2_b
    for i in range(len(feature_name)):
        tab_train2.set_title(i, feature_name[i])
    tab_test.children = wimage_test
    for i in range(len(feature_name)):
        tab_test.set_title(i, feature_name[i])
    tab_util.children = wimage_util
    for i in range(len(feature_name)):
        tab_util.set_title(i, feature_name[i])
    

    
def onclick(b):
    global boston_xtrain,boston_xtest,boston_ytrain,boston_ytest,tab_train,plt
    boston_xtrain, boston_xtest, boston_ytrain, boston_ytest = train_test_split(boston.data[:,:], boston.target, train_size=0.90)
    %matplotlib inline
    %matplotlib inline
    import matplotlib.pyplot as plt
    change.disabled=True
    change_train() 
    display(Javascript('IPython.notebook.execute_cell_range(6,7)'))
    change.disabled=False

change = Button(description = "CHANGER SETS")
change.on_click(onclick)
change_train()

children2.append(tab_train)
general2.children = children2
children3.append(tab_train2)
general3.children = children3
children4.append(tab_test)
general4.children = children4
children5.append(tab_util)
general5.children = children5





# Definition et affichage de l'accordéon principal
general1.children = children1
#general1.set_title(0,"1. Collecte des données")
#general2.set_title(0,"2.1. Séparation set d'entrainement/ set de test")
#general3.set_title(0,"2.2. Entrainement sur 90% des données")
#general4.set_title(0,"3. Test sur 10% des données")
#general5.set_title(0,"4. Utilisation du modèle")


display(HTMLMath(value=r"<h3><b>1. Collecte des données</b></h3>"))
display(HTMLMath(value=r"""On utilise ici un jeu de données sur les logements dans la banlieue de Boston, dans
lequel la valeur d'achat moyenne (donnée cible) et différents paramètres d’entrée (nombre moyen de pièces, âge moyen des logements, pollution, etc...) sont connus pour différentes zones."""))
display(general1)
display(HTMLMath(value=r"<h3><b>2.1. Séparation set d'entrainement / set de test</b></h3>"))
display(HTMLMath(value=r"""Séparation des
données:
</br>90 % pour
le set d’entraînement et 10 % pour le set de test."""))
display(change)
display(general2)
display(HTMLMath(value=r"<h3><b>2.2. Entrainement sur 90% des données</b></h3>"))
display(HTMLMath(value=r"""Représentation de la valeur du logement en fonction d’un paramètre puis modélisation mathématique"""))
display(general3)
display(HTMLMath(value=r"<h3><b>3. Test sur 10% des données</b></h3>"))
display(HTMLMath(value=r"""On teste le modèle obtenu sur un jeu de données de test n'ayant pas servi à la modélisation. Si les résultats sont concluants, on passe à la suite, sinon, on revient à l'étape précédente."""))
display(general4)
display(HTMLMath(value=r"<h3><b>4. Utilisation du modèle</b></h3>"))
display(HTMLMath(value=r"""Connaissant les paramètres d’entrée d’une zone, on peut maintenant utiliser le modèle afin d'estimer la valeur moyenne d'un logement dans cette zone."""))
display(general5)

general1.selected_index = None
general2.selected_index = None
general3.selected_index = None
general4.selected_index = None
general5.selected_index = None





Accordion(children=(Accordion(children=(Textarea(value="TARGET\nMedian value of owner-occupied homes in $1000'…

Accordion(children=(VBox(children=(Button(description='CHANGE SET', style=ButtonStyle()), Tab(children=(Image(…

Accordion(children=(Tab(children=(VBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01…

Accordion(children=(Tab(children=(VBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01…

Accordion(children=(Tab(children=(VBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01…

Accordion(children=(HTMLMath(value='hello'),), _titles={'0': '5. Avec plusieurs paramètres'})

In [ ]:
deuxaxes = HTMLMath(value = r"hello")

def td():
    global boston_xtrain,boston_xtest,boston_ytrain,boston_ytest,tab_train,plt,deuxaxes,fig
    xtrain2 = boston_xtrain[:,[5,12]]
    xtest2 = boston_xtest[:,[5,12]]
    regr2 = linear_model.LinearRegression()
    regr2.fit(xtrain2,boston_ytrain)
    ypred2 = regr2.predict(xtest2)
    
    M1 = np.max(xtrain2[:,0])
    M2 = np.max(xtrain2[:,1])
    m1 = np.min(xtrain2[:,0])
    m2 = np.min(xtrain2[:,1])

    x1 = np.zeros((50*50,))
    for i in range(50):
        x1[50*i:50*i+50] = np.linspace(m1,M1,50)
    x2 = np.zeros((50*50,))
    for i in range(50):
        x2[i::50] = np.linspace(m2,M2,50)
    x3 = np.zeros((2500,2))
    x3[:,0]=x1
    x3[:,1]=x2
    y2 = regr2.predict(x3)

    %matplotlib notebook
    %matplotlib notebook
    import matplotlib.pyplot as plt
    fig = plt.figure("deux axes",figsize=[12,8])
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(xtrain2[:,0],xtrain2[:,1],boston_ytrain, color="gray")
    ax.scatter(xtest2[:,0],xtest2[:,1],boston_ytest,  color='red')
    ax.plot(x1,x2,y2,color="blue")
    ax.set_xlabel('RM')
    ax.set_ylabel('LSTAT')
    ax.set_zlabel('TARGET')
    fig.show()
    deuxaxes.value=r"""Equation du modèle sur données d'entrainement (90%): <b>$$TARGET = {:.3f} \times {} {:+.3f} \times {} {:+.2f}$$</b>
    </br>
    Erreur quadratique moyenne sur données de test (10%) (plus c'est petit, mieux c'est): <b>{:.1f}</b>
        </br>
        $R^2$ sur données de test (10%) (1 est parfait, plus la valeur est haute mieux c'est): <b>{:.2f}</b>
        </br></br>
        Vous pouvez faire tourner la figure ci-dessous pour visualiser les données et le modèle.""".format(regr2.coef_[0],feature_name[5],regr2.coef_[1],feature_name[12],regr2.intercept_,mean_squared_error(boston_ytest, ypred2),r2_score(boston_ytest, ypred2))      

    
display(HTMLMath(value=r"<h3><b>5. Avec plusieurs paramètres</b></h3>"))
display(HTMLMath(value=r"""Il est possible d'utiliser encore plus de paramètres (exemple ci-dessous) dans le modèle ou d'utiliser des fonctions mathématiques autres que des fonctions affines afin de mieux coller aux données expérimentales."""))
display(deuxaxes)
td()